In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test', 'state_labels.csv', 'color_labels.csv', 'train_sentiment', 'test_images', 'train_images', 'train_metadata', 'test_metadata', 'test_sentiment', 'train', 'breed_labels.csv']


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, Embedding
#from keras.layers import Conv2D, MaxPooling2D
from keras import optimizers, regularizers
from keras.utils import to_categorical, plot_model
#from keras.preprocessing.image import ImageDataGenerator

def plot_metrics(history):
	# list all data in history
	print(history.history.keys())
	# summarize history for accuracy
	plt.plot(history.history['acc'])
	plt.plot(history.history['val_acc'])
	plt.title('model accuracy')
	plt.ylabel('accuracy')
	plt.xlabel('epoch')
	plt.legend(['train', 'test'], loc='upper left')
	plt.show()
	# summarize history for loss
	plt.plot(history.history['loss'])
	plt.plot(history.history['val_loss'])
	plt.title('model loss')
	plt.ylabel('loss')
	plt.xlabel('epoch')
	plt.legend(['train', 'test'], loc='upper left')
	plt.show()

train = pd.read_csv('../input/train/train.csv')
test = pd.read_csv('../input/test/test.csv')

#print(train.shape)
#print(train.axes)
#print(train.dtypes)

#print(test.shape)
#print(test.axes)
#print(test.dtypes)

pet_ids = test['PetID'].values

y_train = train['AdoptionSpeed']
y_train = to_categorical(y_train)
num_labels = 5

#x_train = train.drop(columns=['Description','PetID','AdoptionSpeed'])
#x_pred = test.drop(columns=['Description','PetID'])
x_train = train.drop(columns=['Name','RescuerID','Description','PetID','AdoptionSpeed'])
x_pred = test.drop(columns=['Name','RescuerID','Description','PetID'])

# cat or dog, binary
x_train['Type'] = x_train['Type'] - 1 
x_pred['Type'] = x_pred['Type'] - 1


for i, column in enumerate(x_train):
	x_train[column] = x_train[column].astype('float32') / x_train[column].astype('float32').max()
	x_pred[column] = x_pred[column].astype('float32') / x_pred[column].astype('float32').max()

print(x_train.shape)
print(x_pred.shape)

input_size = x_train.shape[1]
print(input_size)

# network parameters
batch_size = 128
hidden_units = 128
dropout = 0.3
data_augmentation = False
epochs = 400
#max_batches = 2 * len(x_train) / batch_size
kernel_regularizer = regularizers.l2(0.0001)

model = Sequential()
model.add(Dense(hidden_units, input_dim=input_size, kernel_regularizer=kernel_regularizer))
model.add(Activation('relu'))
model.add(Dropout(dropout))
model.add(Dense(hidden_units, kernel_regularizer=kernel_regularizer))
model.add(Activation('relu'))
model.add(Dropout(dropout))

model.add(Dense(num_labels))
# this is the output for one-hot vector
model.add(Activation('softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

'''
loss, acc = model.evaluate(x_pred, y_test, batch_size=batch_size)
print("\nTest accuracy: %.1f%%" % (100.0 * acc))
print("Test Loss: %.1f" % (loss))
'''

predicted = model.predict_classes(x_pred)
#predicted_prob = model.predict_proba(x_pred)

f = open("submission.csv","w+")

f.write("PetID,AdoptionSpeed\n")
for i in range(len(x_pred)):
	f.write("%s,%d\n" %(pet_ids[i], predicted[i]))
            
f.close()

#for i in range(len(x_pred)):
	#print(pet_ids[i], predicted[i])
	#print(pet_ids[i], predicted[i], predicted_prob[i])

#plot_metrics(history)


Using TensorFlow backend.


(14993, 19)
(3948, 19)
19
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               2560      
_________________________________________________________________
activation_1 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_2 (Activation)    (None, 128)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 5)            